In [2]:
import torch

a = torch.tensor([500, 600])
a[1]

tensor(600)

In [4]:
frame_log = []
frame_log.append({"cls":1, "loc":[2,2]})
frame_log.append({"cls":0, "loc":[1,4]})
frame_log

[{'cls': 1, 'loc': [2, 2]}, {'cls': 0, 'loc': [1, 4]}]

In [13]:
stream_log = []
stream_log.append(frame_log)
stream_log.append(frame_log)
stream_log[-1]

[{'cls': 1, 'loc': [2, 2]}, {'cls': 0, 'loc': [1, 4]}]

In [11]:
len(stream_log)

2